## TYTAN tutorial 追加問題3（カクラス）

最終更新：2023年6月25日 by ビネクラ安田

取り組み方

*   Google Colabで取り組む場合：ファイル＞ドライブにコピーを保存　<font color="red">※このファイルを直接編集しても保存されません</font>
*   Jupyter Notebookに移す場合：ファイル＞ダウンロード＞.ipynbをダウンロード

参考リンク１

*   [TYTANチュートリアル一覧](https://github.com/tytansdk/tytan_tutorial)
*   [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)

出展

*   [量子アニーリング（QUBO）でカクラス（Kakurasu）を解く](https://vigne-cla.com/21-17/)


### 問題
カクラス（Kakurasu）は掛け算の計算が <s>面倒な</s> 必要な変わったパズル。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-17_1.png" width = 20%>
</div>

答えはこちら。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-17_2.png" width = 20%>
</div>

上辺・左辺の小さな数字が各列のマスの重みを表す。白マスを０、黒マスを１と考えて、各マスの重みをかけて合計した数値が右辺・下辺の数字と一致するように塗る。例えば、一番上の行は １×１＋２×０＋３×０＋４×１＝５ を意味している（赤字がマスの色に対応）。

ブラウザ上で遊んでみると良い → [Puzzle Team](https://ja.puzzle-kakurasu.com/)

### このQUBO設定を使おう

<font color="red">「解が0と1だけの方程式」</font>

「重さ1, 2, 3のボールのどれを取れば合計3の重さになるか？」のは次のように設定できる。
```
H = (1*q0 + 2*q1 + 3*q2 - 3)**2
```

解は [q0, q1, q2] = [0, 0, 1] or [1, 1, 0] となり、1, 2のボール、または3のボールを取れば良いという結果になる。

その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 条件設定

16個の量子ビットを各マスに対応させる。結果が１なら黒マス（＝重み有効）と考える。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-17_3-300x293.png" width = 20%>
</div>

一行だけ取り出して考える。この行は「1×q0＋2×q1＋3×q2＋4×q3＝5」にしたい。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-17_4-768x120.png" width = 50%>
</div>

ここで基本の条件設定を思い浮かべる。「４個の量子ビットからｍ個を１にする」は次の式。

```
H = (q0 + q1 + q2 + q3 - m)**2
```

これはつまり、**４個の量子ビットを足した値がｍになる**ということ。合計値がｍであればエネルギーは最小で、合計値がｍからずれるとエネルギーが高くなる。

ここで、４つの量子ビットを足す際に係数をかける。

```
H = (1*q0 + 2*q1 + 3*q2 + 4*q3 - 5)**2
```

実はこれで式が完成。**「1×q0＋2×q1＋3×q2＋4×q3」の部分が５のときにエネルギーは最小**となり、５からずれるとエネルギーが高くなる。少し柔軟な発想が必要だ。

## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [4]:
from tytan import *
import numpy as np

#量子ビットを用意する
q00 = symbols('q00')
q01 = symbols('q01')
q02 = symbols('q02')
q03 = symbols('q03')
q04 = symbols('q04')
q05 = symbols('q05')
q06 = symbols('q06')
q07 = symbols('q07')
q08 = symbols('q08')
q09 = symbols('q09')
q10 = symbols('q10')
q11 = symbols('q11')
q12 = symbols('q12')
q13 = symbols('q13')
q14 = symbols('q14')
q15 = symbols('q15')

#各行、「重み付き合計値が指定の値になる」
H = 0
H += (1*q00 + 2*q01 + 3*q02 + 4*q03 - 5)**2
H += (1*q04 + 2*q05 + 3*q06 + 4*q07 - 7)**2
H += (1*q08 + 2*q09 + 3*q10 + 4*q11 - 5)**2
H += (1*q12 + 2*q13 + 3*q14 + 4*q15 - 8)**2

#各列、「重み付き合計値が指定の値になる」
H += (1*q00 + 2*q04 + 3*q08 + 4*q12 - 5)**2
H += (1*q01 + 2*q05 + 3*q09 + 4*q13 - 3)**2
H += (1*q02 + 2*q06 + 3*q10 + 4*q14 - 9)**2
H += (1*q03 + 2*q07 + 3*q11 + 4*q15 - 7)**2


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo)

#上位5件
for r in result[:5]:
    print(r)
    print(np.array(list(r[0].values())).reshape(4, 4))

offset
327
[{'q00': 1, 'q01': 0, 'q02': 0, 'q03': 1, 'q04': 0, 'q05': 0, 'q06': 1, 'q07': 1, 'q08': 0, 'q09': 1, 'q10': 1, 'q11': 0, 'q12': 1, 'q13': 0, 'q14': 1, 'q15': 1}, -327.0, 61]
[[1 0 0 1]
 [0 0 1 1]
 [0 1 1 0]
 [1 0 1 1]]
[{'q00': 1, 'q01': 0, 'q02': 0, 'q03': 1, 'q04': 0, 'q05': 0, 'q06': 1, 'q07': 1, 'q08': 1, 'q09': 1, 'q10': 1, 'q11': 0, 'q12': 0, 'q13': 0, 'q14': 1, 'q15': 1}, -324.0, 12]
[[1 0 0 1]
 [0 0 1 1]
 [1 1 1 0]
 [0 0 1 1]]
[{'q00': 1, 'q01': 0, 'q02': 1, 'q03': 0, 'q04': 0, 'q05': 0, 'q06': 1, 'q07': 1, 'q08': 0, 'q09': 1, 'q10': 1, 'q11': 0, 'q12': 1, 'q13': 0, 'q14': 1, 'q15': 1}, -324.0, 7]
[[1 0 1 0]
 [0 0 1 1]
 [0 1 1 0]
 [1 0 1 1]]
[{'q00': 0, 'q01': 1, 'q02': 1, 'q03': 0, 'q04': 1, 'q05': 1, 'q06': 0, 'q07': 1, 'q08': 1, 'q09': 0, 'q10': 1, 'q11': 0, 'q12': 0, 'q13': 0, 'q14': 1, 'q15': 1}, -323.0, 16]
[[0 1 1 0]
 [1 1 0 1]
 [1 0 1 0]
 [0 0 1 1]]
[{'q00': 0, 'q01': 1, 'q02': 1, 'q03': 0, 'q04': 1, 'q05': 1, 'q06': 1, 'q07': 0, 'q08': 1, 'q09': 0, 'q10': 0

一番目の解は模範解答の通りである。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-17_2.png" width = 20%>
</div>